In [1]:
# !unzip /home/jupyter-st124895/cv_project/04_experiments.zip

In [1]:
!pwd

/home/jupyter-st124895/cv_project


In [2]:
!nvidia-smi

Fri Nov 21 07:07:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off |   00000000:84:00.0 Off |                  N/A |
| 48%   66C    P2             70W /  250W |     896MiB /  11264MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# ==============================================================================
# SECTION 1: Mount Google Drive & GPU Check
# ==============================================================================

# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# Check GPU
import subprocess
subprocess.run(['nvidia-smi'])

# Install dependencies
subprocess.run(['pip', 'install', 'torch', 'torchvision', 'pandas', 'matplotlib',
                'seaborn', 'pycocotools', 'albumentations', '-q'])

import torch
import torchvision
print(f"[OK] PyTorch: {torch.__version__}")
print(f"[OK] Torchvision: {torchvision.__version__}")
print(f"[OK] CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"[OK] GPU: {torch.cuda.get_device_name(1)}")

Fri Nov 21 07:07:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off |   00000000:84:00.0 Off |                  N/A |
| 48%   66C    P2             70W /  250W |     896MiB /  11264MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


[OK] PyTorch: 2.8.0+cu128
[OK] Torchvision: 0.23.0+cu128
[OK] CUDA available: True
[OK] GPU: NVIDIA GeForce RTX 2080 Ti


In [4]:
# ==============================================================================
# SECTION 2: Dataset & Model Classes
# ==============================================================================

from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN
# For ResNet101 backbone
from torchvision.models import resnet101, ResNet101_Weights
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection.anchor_utils import AnchorGenerator
from torch.utils.data import Dataset, DataLoader
import json
import cv2
# import numpy as np
from pathlib import Path
import pandas as pd
from datetime import datetime
import time
from typing import Dict, List, Optional
import matplotlib.pyplot as plt
import seaborn as sns
import albumentations as A
from albumentations.pytorch import ToTensorV2


class COCODataset(Dataset):
    """
    Custom dataset for loading COCO format annotations with Albumentations support.
    """
    def __init__(self, root_dir, annotation_file, transforms=None):
        """
        Args:
            root_dir: Path to images directory
            annotation_file: Path to COCO JSON annotation file
            transforms: Optional Albumentations transforms to apply
        """
        self.root_dir = Path(root_dir)
        self.transforms = transforms

        # Load COCO annotations
        with open(annotation_file, 'r') as f:
            self.coco_data = json.load(f)

        # Create image id to annotations mapping
        self.image_id_to_anns = {}
        for ann in self.coco_data['annotations']:
            img_id = ann['image_id']
            if img_id not in self.image_id_to_anns:
                self.image_id_to_anns[img_id] = []
            self.image_id_to_anns[img_id].append(ann)

        # Get list of images
        self.images = self.coco_data['images']

        print(f"[OK] Loaded {len(self.images)} images")
        print(f"[OK] Loaded {len(self.coco_data['annotations'])} annotations")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Get image info
        img_info = self.images[idx]
        img_id = img_info['id']
        img_path = self.root_dir / img_info['file_name']

        # Load image as numpy array (for Albumentations)
        image = cv2.imread(str(img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Get annotations for this image
        anns = self.image_id_to_anns.get(img_id, [])

        # Prepare boxes and labels for Albumentations
        boxes = []
        labels = []
        areas = []

        for ann in anns:
            # COCO format: [x, y, width, height]
            x, y, w, h = ann['bbox']
            # Convert to [x1, y1, x2, y2] for Albumentations
            boxes.append([x, y, x + w, y + h])
            labels.append(ann['category_id'])
            areas.append(ann['area'])

        # Apply Albumentations transforms
        if self.transforms and len(boxes) > 0:
            try:
                transformed = self.transforms(
                    image=image,
                    bboxes=boxes,
                    labels=labels
                )
                image = transformed['image']
                boxes = transformed['bboxes']
                labels = transformed['labels']
            except Exception as e:
                # If transformation fails, use original data
                print(f"[WARNING] Transform failed for image {img_id}: {e}")

        # Always convert image to tensor at the end
        if not isinstance(image, torch.Tensor):
            image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0

        # Convert to tensors
        if len(boxes) > 0:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)

            # Recalculate areas after transforms
            if len(boxes) > 0:
                areas = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
            else:
                areas = torch.as_tensor(areas, dtype=torch.float32)
        else:
            # Empty annotations
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
            areas = torch.zeros((0,), dtype=torch.float32)

        image_id = torch.tensor([img_id])

        # Create target dict
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = areas
        target["iscrowd"] = torch.zeros((len(boxes),), dtype=torch.int64)

        return image, target


def get_train_transforms(config):
    """
    Create training transforms with augmentation based on config.

    Args:
        config: Experiment configuration dict with augmentation parameters

    Returns:
        Albumentations Compose object
    """
    transforms_list = []

    # Color augmentation
    if config.get('hsv_h', 0) > 0 or config.get('hsv_s', 0) > 0 or config.get('hsv_v', 0) > 0:
        transforms_list.append(
            A.HueSaturationValue(
                hue_shift_limit=int(config.get('hsv_h', 0) * 100),
                sat_shift_limit=int(config.get('hsv_s', 0) * 100),
                val_shift_limit=int(config.get('hsv_v', 0) * 100),
                p=0.9
            )
        )

    # Geometric augmentation
    if config.get('degrees', 0) > 0 or config.get('translate', 0) > 0 or config.get('scale', 0) > 0:
        transforms_list.append(
            A.ShiftScaleRotate(
                shift_limit=config.get('translate', 0.0),
                scale_limit=config.get('scale', 0.0),
                rotate_limit=config.get('degrees', 0),
                border_mode=cv2.BORDER_CONSTANT,
                p=0.5
            )
        )

    # Horizontal flip
    if config.get('horizontal_flip', 0.0) > 0:
        transforms_list.append(
            A.HorizontalFlip(p=config.get('horizontal_flip', 0.0))
        )

    # Vertical flip
    if config.get('vertical_flip', 0.0) > 0:
        transforms_list.append(
            A.VerticalFlip(p=config.get('vertical_flip', 0.0))
        )

    # Blur
    if config.get('blur', False):
        transforms_list.append(
            A.Blur(blur_limit=3, p=0.1)
        )

    # Brightness/Contrast
    if config.get('brightness_contrast', False):
        transforms_list.append(
            A.RandomBrightnessContrast(
                brightness_limit=0.2,
                contrast_limit=0.2,
                p=0.5
            )
        )

    # Create compose with bbox parameters
    return A.Compose(
        transforms_list,
        bbox_params=A.BboxParams(
            format='pascal_voc',  # [x1, y1, x2, y2]
            label_fields=['labels'],
            min_visibility=0.3,  # Keep boxes with at least 30% visible
            min_area=100.0  # Keep boxes with at least 100 pixels area
        )
    )


def get_val_transforms():
    """
    Create validation transforms (no augmentation, just normalization).

    Returns:
        Albumentations Compose object
    """
    return A.Compose(
        [],  # No augmentation for validation
        bbox_params=A.BboxParams(
            format='pascal_voc',
            label_fields=['labels']
        )
    )


def get_model(num_classes, backbone='resnet50', pretrained=True,
              # Configurable architecture parameters (Opción A):
              box_score_thresh=None,
              box_nms_thresh=None,
              box_detections_per_img=None,
              rpn_fg_iou_thresh=None,
              rpn_bg_iou_thresh=None,
              box_positive_fraction=None):
    """
    Create Faster R-CNN model with configurable architecture parameters.

    Args:
        num_classes: Number of classes (including background)
        backbone: Backbone architecture ('resnet50' or 'resnet101')
                 - resnet50: 41.3M params, faster training (~2-3 hrs for 110 epochs)
                 - resnet101: 60.2M params, better accuracy (+2-5% expected), slower (~3-4 hrs)
        pretrained: Whether to use pretrained weights (recommended: True)

        # Configurable parameters (None = use torchvision defaults):
        box_score_thresh: Minimum confidence for detection (default: 0.05)
        box_nms_thresh: NMS IoU threshold (default: 0.5)
        box_detections_per_img: Max detections per image (default: 100)
        rpn_fg_iou_thresh: RPN foreground IoU threshold (default: 0.7)
        rpn_bg_iou_thresh: RPN background IoU threshold (default: 0.3)
        box_positive_fraction: Positive sample ratio in ROI head (default: 0.25)

    Returns:
        model: Faster R-CNN model

    Example usage in experiments_config:
        {
            'name': 'resnet101_best_params',
            'backbone': 'resnet101',  # Use more powerful backbone
            'box_score_thresh': 0.10,
            'box_positive_fraction': 0.30,
            ...
        }
    """
    # Set defaults for configurable parameters (match torchvision defaults)
    box_score_thresh = box_score_thresh if box_score_thresh is not None else 0.05
    box_nms_thresh = box_nms_thresh if box_nms_thresh is not None else 0.5
    box_detections_per_img = box_detections_per_img if box_detections_per_img is not None else 100
    rpn_fg_iou_thresh = rpn_fg_iou_thresh if rpn_fg_iou_thresh is not None else 0.7
    rpn_bg_iou_thresh = rpn_bg_iou_thresh if rpn_bg_iou_thresh is not None else 0.3
    box_positive_fraction = box_positive_fraction if box_positive_fraction is not None else 0.25

    # Load pretrained model based on backbone choice
    if backbone == 'resnet50':
        if pretrained:
            model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
        else:
            model = fasterrcnn_resnet50_fpn(weights=None)

    elif backbone == 'resnet101':
        # Create ResNet101 backbone with FPN
        backbone_model = resnet_fpn_backbone(
            backbone_name='resnet101',
            weights=ResNet101_Weights.DEFAULT if pretrained else None,
            trainable_layers=5
        )

        # Create anchor generator (same as default FasterRCNN)
        anchor_generator = AnchorGenerator(
            sizes=((32,), (64,), (128,), (256,), (512,)),
            aspect_ratios=((0.5, 1.0, 2.0),) * 5
        )

        # Create ROI pooler (same config as default)
        roi_pooler = torchvision.ops.MultiScaleRoIAlign(
            featmap_names=['0', '1', '2', '3'],
            output_size=7,
            sampling_ratio=2
        )

        # Create Faster R-CNN model with ResNet101 backbone
        model = FasterRCNN(
            backbone_model,
            num_classes=91,  # Will be replaced below
            rpn_anchor_generator=anchor_generator,
            box_roi_pool=roi_pooler
        )

    else:
        raise ValueError(f"Backbone {backbone} not supported. Use 'resnet50' or 'resnet101'")

    # Replace the classifier head
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # Apply configurable parameters to the model
    model.roi_heads.score_thresh = box_score_thresh
    model.roi_heads.nms_thresh = box_nms_thresh
    model.roi_heads.detections_per_img = box_detections_per_img
    model.roi_heads.fg_iou_thresh = rpn_fg_iou_thresh
    model.roi_heads.bg_iou_thresh = rpn_bg_iou_thresh
    model.roi_heads.positive_fraction = box_positive_fraction

    # Also update RPN thresholds
    model.rpn.fg_iou_thresh = rpn_fg_iou_thresh
    model.rpn.bg_iou_thresh = rpn_bg_iou_thresh

    return model


In [5]:
# ==============================================================================
# SECTION 3: ExperimentTrainer Class
# ==============================================================================

class FasterRCNNTrainer:
    """
    Manages Faster R-CNN training experiments with automatic logging and tracking.
    """

    def __init__(self, experiment_name: str, model_family_dir: str = '02_faster_rcnn'):
        """
        Initialize experiment trainer.

        Args:
            experiment_name: Name for this experiment
            model_family_dir: Directory name for model family (e.g., '02_faster_rcnn')
        """
        self.experiment_name = experiment_name
        self.model_family_dir = Path(EXPERIMENTS_PATH) / model_family_dir
        self.model_family_dir.mkdir(parents=True, exist_ok=True)

        # Auto-increment experiment ID
        self.experiment_id = self._get_next_experiment_id()
        self.experiment_dir = self.model_family_dir / self.experiment_id
        self.experiment_dir.mkdir(parents=True, exist_ok=True)

        # Create subdirectories
        self.weights_dir = self.experiment_dir / 'weights'
        self.plots_dir = self.experiment_dir / 'plots'
        self.weights_dir.mkdir(exist_ok=True)
        self.plots_dir.mkdir(exist_ok=True)

        print(f"\n{'='*70}")
        print(f"  EXPERIMENT: {self.experiment_id}")
        print(f"  Family: {model_family_dir}")
        print(f"  Directory: {self.experiment_dir}")
        print(f"{'='*70}\n")

    def _get_next_experiment_id(self) -> str:
        """Auto-increment experiment number"""
        existing_experiments = list(self.model_family_dir.glob('exp_*'))

        if not existing_experiments:
            exp_num = 1
        else:
            numbers = []
            for exp_path in existing_experiments:
                try:
                    num_str = exp_path.name.split('_')[1]
                    numbers.append(int(num_str))
                except (IndexError, ValueError):
                    continue

            exp_num = max(numbers) + 1 if numbers else 1

        return f"exp_{exp_num:03d}_{self.experiment_name}"

    def _get_dataset_info(self, train_ann_file: str, val_ann_file: str) -> Dict:
        """Extract dataset information from COCO JSON files"""
        dataset_info = {
            'num_train_images': 0,
            'num_val_images': 0,
            'num_train_boxes': 0,
            'num_val_boxes': 0
        }

        try:
            # Load train annotations
            with open(train_ann_file, 'r') as f:
                train_data = json.load(f)
            dataset_info['num_train_images'] = len(train_data['images'])
            dataset_info['num_train_boxes'] = len(train_data['annotations'])

            # Load val annotations
            with open(val_ann_file, 'r') as f:
                val_data = json.load(f)
            dataset_info['num_val_images'] = len(val_data['images'])
            dataset_info['num_val_boxes'] = len(val_data['annotations'])

        except Exception as e:
            print(f"[WARNING] Could not extract dataset info: {e}")

        return dataset_info

    def train(self, train_loader, val_loader, model, optimizer, lr_scheduler,
              num_epochs, device, config):
        """
        Train Faster R-CNN model.

        Args:
            train_loader: Training data loader
            val_loader: Validation data loader
            model: Faster R-CNN model
            optimizer: Optimizer
            lr_scheduler: Learning rate scheduler
            num_epochs: Number of epochs
            device: Device to train on
            config: Experiment configuration dict
        """
        print(f"[INFO] Starting training for {num_epochs} epochs...")
        print(f"[INFO] Device: {device}")

        model.to(device)

        # Training history
        history = {
            'train_loss': [],
            'val_loss': [],
            'learning_rate': []
        }

        best_val_loss = float('inf')
        best_epoch = 0
        patience_counter = 0

        start_time = time.time()

        for epoch in range(num_epochs):
            epoch_start = time.time()

            # Training phase
            model.train()
            train_loss = 0.0

            print(f"\n[EPOCH {epoch+1}/{num_epochs}]")

            for batch_idx, (images, targets) in enumerate(train_loader):
                # Move to device
                images = [img.to(device) for img in images]
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                # Forward pass
                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())

                # Backward pass
                optimizer.zero_grad()
                losses.backward()
                optimizer.step()

                train_loss += losses.item()

                if (batch_idx + 1) % 10 == 0:
                    print(f"  Batch [{batch_idx+1}/{len(train_loader)}] - Loss: {losses.item():.4f}")

            train_loss /= len(train_loader)
            history['train_loss'].append(train_loss)
            history['learning_rate'].append(optimizer.param_groups[0]['lr'])

            # Validation phase
            # Note: Faster R-CNN requires training mode to compute losses
            # but we use no_grad() to prevent gradient computation
            model.train()  # Keep in train mode to get loss_dict
            val_loss = 0.0

            with torch.no_grad():
                for images, targets in val_loader:
                    images = [img.to(device) for img in images]
                    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                    # Forward pass - model returns loss_dict when in train mode
                    loss_dict = model(images, targets)
                    losses = sum(loss for loss in loss_dict.values())
                    val_loss += losses.item()

            val_loss /= len(val_loader)
            history['val_loss'].append(val_loss)

            epoch_time = time.time() - epoch_start

            print(f"  Train Loss: {train_loss:.4f}")
            print(f"  Val Loss: {val_loss:.4f}")
            print(f"  Time: {epoch_time:.2f}s")

            # Save best model
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_epoch = epoch + 1
                torch.save(model.state_dict(), self.weights_dir / 'best.pt')
                print(f"  ✓ Best model saved (val_loss: {val_loss:.4f})")
                patience_counter = 0
            else:
                patience_counter += 1

            # Save last model
            torch.save(model.state_dict(), self.weights_dir / 'last.pt')

            # Step learning rate scheduler
            lr_scheduler.step()

            # Early stopping
            if config.get('patience', 0) > 0 and patience_counter >= config['patience']:
                print(f"\n[EARLY STOPPING] No improvement for {config['patience']} epochs")
                break

        total_time = time.time() - start_time
        self.training_time = total_time  # Save as instance attribute for later use

        print(f"\n{'='*70}")
        print(f"  TRAINING COMPLETED")
        print(f"  Best Epoch: {best_epoch}")
        print(f"  Best Val Loss: {best_val_loss:.4f}")
        print(f"  Total Time: {total_time/60:.2f} minutes")
        print(f"{'='*70}\n")

        # Save training history
        self._save_history(history, best_epoch, total_time, config)

        return history, best_val_loss, best_epoch

    def _save_history(self, history, best_epoch, training_time, config):
        """Save training history and plots with unified style"""
        # Unified plot configuration (matching YOLOv8)
        fig, axes = plt.subplots(1, 2, figsize=(18, 10))

        # Add suptitle
        fig.suptitle(f'Training Results - {self.experiment_id}',
                     fontsize=16, fontweight='bold')

        # Loss plot (using YOLOv8 color scheme)
        axes[0].plot(history['train_loss'], label='Train Loss',
                     linewidth=2, color='#3498db')  # Blue
        axes[0].plot(history['val_loss'], label='Val Loss',
                     linewidth=2, color='#e74c3c')  # Red
        axes[0].axvline(x=best_epoch-1, color='#e74c3c', linestyle='--',
                       label=f'Best Epoch ({best_epoch})')
        axes[0].set_xlabel('Epoch', fontsize=12)
        axes[0].set_ylabel('Loss', fontsize=12)
        axes[0].set_title('Training & Validation Loss', fontsize=14)
        axes[0].legend(fontsize=10)
        axes[0].grid(True, alpha=0.3)

        # Learning rate plot (using YOLOv8 color scheme)
        axes[1].plot(history['learning_rate'], linewidth=2,
                     color='#16a085')  # Teal
        axes[1].set_xlabel('Epoch', fontsize=12)
        axes[1].set_ylabel('Learning Rate', fontsize=12)
        axes[1].set_title('Learning Rate Schedule', fontsize=14)
        axes[1].set_yscale('log')  # Log scale for LR
        axes[1].grid(True, alpha=0.3)

        plt.tight_layout(rect=[0, 0, 1, 0.96])  # Leave space for suptitle
        plt.savefig(self.plots_dir / 'training_history.png',
                    dpi=300, bbox_inches='tight')  # High DPI
        plt.close()

        # Save experiment log (with history for plots)
        log = {
            'experiment_id': self.experiment_id,
            'experiment_name': self.experiment_name,
            'best_epoch': best_epoch,
            'best_val_loss': history['val_loss'][best_epoch-1],
            'final_train_loss': history['train_loss'][-1],
            'final_val_loss': history['val_loss'][-1],
            'training_time_minutes': training_time / 60,
            'config': config,
            'timestamp': datetime.now().isoformat(),
            # Add history for comprehensive plots in SECTION 6
            'train_loss': history['train_loss'],
            'val_loss': history['val_loss'],
            'learning_rate': history['learning_rate']
        }

        with open(self.experiment_dir / 'experiment_log.json', 'w') as f:
            json.dump(log, f, indent=2)

        print(f"[OK] Training history saved to {self.plots_dir}")


# ------------------------------------------------------------------------------
# Evaluation Functions (COCO Metrics)
# ------------------------------------------------------------------------------

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import tempfile

def evaluate_model_coco(model, data_loader, device, ann_file):
    """
    Evaluate Faster R-CNN model using COCO metrics.

    Args:
        model: Trained Faster R-CNN model
        data_loader: DataLoader for evaluation
        device: Device to run evaluation on
        ann_file: Path to COCO annotation file

    Returns:
        dict: Dictionary with mAP metrics
    """
    model.eval()
    model.to(device)

    # Collect all predictions
    coco_results = []

    print("[INFO] Running inference for COCO evaluation...")
    with torch.no_grad():
        for images, targets in data_loader:
            images = [img.to(device) for img in images]
            predictions = model(images)

            # Convert predictions to COCO format
            for pred, target in zip(predictions, targets):
                image_id = target['image_id'].item()
                boxes = pred['boxes'].cpu().numpy()
                scores = pred['scores'].cpu().numpy()
                labels = pred['labels'].cpu().numpy()

                # Convert boxes from [x1, y1, x2, y2] to COCO format [x, y, w, h]
                for box, score, label in zip(boxes, scores, labels):
                    x1, y1, x2, y2 = box
                    coco_results.append({
                        'image_id': image_id,
                        'category_id': int(label),
                        'bbox': [float(x1), float(y1), float(x2 - x1), float(y2 - y1)],
                        'score': float(score)
                    })

    print(f"[OK] Generated {len(coco_results)} predictions")

    # Load COCO ground truth and evaluate
    coco_gt = COCO(str(ann_file))

    # Save predictions to temporary file
    with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
        json.dump(coco_results, f)
        results_file = f.name

    coco_dt = coco_gt.loadRes(results_file)

    # Run COCO evaluation
    print("\n[INFO] Running COCO evaluation...")
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    # Extract metrics
    metrics = {
        'mAP_50_95': coco_eval.stats[0],
        'mAP_50': coco_eval.stats[1],
        'mAP_75': coco_eval.stats[2],
        'precision': coco_eval.stats[1],  # Use mAP@50 as precision proxy
        'recall': coco_eval.stats[8],     # Use AR@100 as recall proxy
        'f1_score': 2 * (coco_eval.stats[1] * coco_eval.stats[8]) / (coco_eval.stats[1] + coco_eval.stats[8] + 1e-6)
    }

    # Measure inference time
    print("\n[INFO] Measuring inference time...")
    import time
    inference_times = []

    model.eval()
    with torch.no_grad():
        # Warm-up
        for images, _ in data_loader:
            images = [img.to(device) for img in images]
            _ = model(images)
            break

        # Measure
        for images, _ in data_loader:
            images = [img.to(device) for img in images]
            start = time.time()
            _ = model(images)
            end = time.time()
            inference_times.append((end - start) * 1000 / len(images))  # ms per image

    metrics['inference_time_ms'] = sum(inference_times) / len(inference_times) if inference_times else 0.0

    # Calculate total parameters
    total_params = sum(p.numel() for p in model.parameters())
    metrics['total_params_M'] = total_params / 1e6

    print(f"[OK] Inference time: {metrics['inference_time_ms']:.2f} ms/image")
    print(f"[OK] Total parameters: {metrics['total_params_M']:.2f}M")

    # Clean up
    import os
    os.unlink(results_file)

    return metrics


def save_metrics_to_csv(experiment_dir, experiment_id, experiment_name, config,
                        metrics, dataset_info, training_time, best_epoch):
    """Save experiment metrics to CSV files (master + family)."""

    # Prepare row data (aligned with YOLOv8 format)
    row_data = {
        'experiment_id': experiment_id,
        'experiment_name': experiment_name,  # Renamed from 'name' for consistency
        'model_family': '02_faster_rcnn',
        'model_variant': config.get('backbone', 'resnet50'),  # Renamed from 'backbone'
        'timestamp': datetime.now().isoformat(),
        'dataset_version': 'v1',  # Added for consistency
        'num_train_images': dataset_info.get('num_train_images', 0),
        'num_val_images': dataset_info.get('num_val_images', 0),
        'num_train_boxes': dataset_info.get('num_train_boxes', 0),
        'num_val_boxes': dataset_info.get('num_val_boxes', 0),
        'num_epochs': config.get('num_epochs', 50),
        'batch_size': config.get('batch_size', 4),
        'img_size': 800,  # Faster R-CNN default input size
        'mAP_50': metrics.get('mAP_50', 0.0),
        'mAP_50_95': metrics.get('mAP_50_95', 0.0),
        'mAP_75': metrics.get('mAP_75', 0.0),
        'precision': metrics.get('precision', 0.0),
        'recall': metrics.get('recall', 0.0),
        'f1_score': metrics.get('f1_score', 0.0),
        'best_epoch': best_epoch,  # Added for tracking
        'inference_time_ms': metrics.get('inference_time_ms', 0.0),  # Added
        'training_time_hours': training_time / 3600,  # Changed from minutes to hours
        'total_params_M': metrics.get('total_params_M', 0.0),  # Added
        # Hyperparameters (extra info for Faster R-CNN)
        'pretrained': config.get('pretrained', True),
        'learning_rate': config.get('learning_rate', 0.005),
        'momentum': config.get('momentum', 0.9),
        'weight_decay': config.get('weight_decay', 0.0005),
        'step_size': config.get('step_size', 10),
        'gamma': config.get('gamma', 0.1),
        'patience': config.get('patience', 10),
        # Augmentation params
        'hsv_h': config.get('hsv_h', 0.0),
        'hsv_s': config.get('hsv_s', 0.0),
        'hsv_v': config.get('hsv_v', 0.0),
        'degrees': config.get('degrees', 0.0),
        'translate': config.get('translate', 0.0),
        'scale': config.get('scale', 0.0),
        'horizontal_flip': config.get('horizontal_flip', 0.0),
        'vertical_flip': config.get('vertical_flip', 0.0),
        'blur': config.get('blur', False),
        'brightness_contrast': config.get('brightness_contrast', False),
        # Architecture params (Opción A)
        'box_score_thresh': config.get('box_score_thresh', 0.05),
        'box_nms_thresh': config.get('box_nms_thresh', 0.5),
        'box_detections_per_img': config.get('box_detections_per_img', 100),
        'rpn_fg_iou_thresh': config.get('rpn_fg_iou_thresh', 0.7),
        'rpn_bg_iou_thresh': config.get('rpn_bg_iou_thresh', 0.3),
        'box_positive_fraction': config.get('box_positive_fraction', 0.25)
    }

    # Save to family CSV
    family_csv = Path(EXPERIMENTS_PATH) / '02_faster_rcnn' / '02_faster_rcnn_experiments.csv'
    df_row = pd.DataFrame([row_data])

    if family_csv.exists():
        df_existing = pd.read_csv(family_csv)
        df_combined = pd.concat([df_existing, df_row], ignore_index=True)
        df_combined.to_csv(family_csv, index=False)
    else:
        df_row.to_csv(family_csv, index=False)

    print(f"[OK] Metrics saved to: {family_csv}")

    # Save to master CSV
    master_csv = Path(EXPERIMENTS_PATH) / 'all_experiments_log.csv'

    if master_csv.exists():
        df_existing = pd.read_csv(master_csv)
        df_combined = pd.concat([df_existing, df_row], ignore_index=True)
        df_combined.to_csv(master_csv, index=False)
    else:
        df_row.to_csv(master_csv, index=False)

    print(f"[OK] Metrics saved to: {master_csv}")


def update_best_model_tracker(experiment_id, experiment_name, metrics, config):
    """
    Update best model JSON files (overall + family).

    Uses mAP@0.5:0.95 as criterion (standard COCO metric), same as YOLOv8.
    """

    family_best = Path(EXPERIMENTS_PATH) / '02_faster_rcnn' / '02_faster_rcnn_best_model.json'

    best_data = {
        'experiment_id': experiment_id,
        'experiment_name': experiment_name,
        'mAP_50': metrics['mAP_50'],
        'mAP_50_95': metrics['mAP_50_95'],
        'config': config,
        'timestamp': datetime.now().isoformat()
    }

    # Update family best (using mAP@0.5:0.95 as criterion)
    update_family = True
    if family_best.exists():
        with open(family_best, 'r') as f:
            current_best = json.load(f)
        if current_best.get('mAP_50_95', 0.0) >= metrics['mAP_50_95']:
            update_family = False

    if update_family:
        with open(family_best, 'w') as f:
            json.dump(best_data, f, indent=2)
        print(f"[OK] Updated best model for 02_faster_rcnn: {experiment_id} (mAP@0.5:0.95 = {metrics['mAP_50_95']:.4f})")

    # Update overall best (using mAP@0.5:0.95 as criterion, same as YOLOv8)
    overall_best = Path(EXPERIMENTS_PATH) / 'best_model_overall.json'

    update_overall = True
    if overall_best.exists():
        with open(overall_best, 'r') as f:
            current_best = json.load(f)
        if current_best.get('mAP_50_95', 0.0) >= metrics['mAP_50_95']:
            update_overall = False

    if update_overall:
        with open(overall_best, 'w') as f:
            json.dump(best_data, f, indent=2)
        print(f"[OK] Updated overall best model: {experiment_id} (mAP@0.5:0.95 = {metrics['mAP_50_95']:.4f})")


In [6]:
# ==============================================================================
# SECTION 4: Helper Functions for Evaluation & Plotting
# ==============================================================================

def create_comprehensive_plots(experiment_dir, history, metrics, best_epoch, experiment_id):
    """
    Create comprehensive training plots (6 subplots like YOLOv8).

    Args:
        experiment_dir: Path to experiment directory
        history: Training history dict
        metrics: Evaluation metrics dict
        best_epoch: Best epoch number
        experiment_id: Experiment ID string
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()

    # Suptitle
    fig.suptitle(f'Comprehensive Training Results - {experiment_id}',
                 fontsize=16, fontweight='bold')

    # 1. Loss curves
    epochs = range(1, len(history['train_loss']) + 1)
    axes[0].plot(epochs, history['train_loss'], label='Train Loss',
                 linewidth=2, color='#3498db')
    axes[0].plot(epochs, history['val_loss'], label='Val Loss',
                 linewidth=2, color='#e74c3c')
    axes[0].axvline(x=best_epoch, color='#e74c3c', linestyle='--',
                   label=f'Best Epoch ({best_epoch})', alpha=0.7)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title('Training & Validation Loss', fontsize=14)
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)

    # 2. mAP metrics (final values as bars)
    map_metrics = ['mAP@0.5', 'mAP@0.5:0.95', 'mAP@0.75']
    map_values = [metrics.get('mAP_50', 0), metrics.get('mAP_50_95', 0),
                  metrics.get('mAP_75', 0)]
    colors_map = ['#2ecc71', '#3498db', '#9b59b6']
    axes[1].bar(map_metrics, map_values, color=colors_map, alpha=0.8, edgecolor='black')
    axes[1].set_ylabel('Score', fontsize=12)
    axes[1].set_title('mAP Metrics', fontsize=14)
    axes[1].set_ylim([0, 1])
    axes[1].grid(True, alpha=0.3, axis='y')
    # Add value labels on bars
    for i, v in enumerate(map_values):
        axes[1].text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=10, fontweight='bold')

    # 3. Precision & Recall
    pr_metrics = ['Precision', 'Recall']
    pr_values = [metrics.get('precision', 0), metrics.get('recall', 0)]
    colors_pr = ['#e74c3c', '#9b59b6']
    axes[2].bar(pr_metrics, pr_values, color=colors_pr, alpha=0.8, edgecolor='black')
    axes[2].set_ylabel('Score', fontsize=12)
    axes[2].set_title('Precision & Recall', fontsize=14)
    axes[2].set_ylim([0, 1])
    axes[2].grid(True, alpha=0.3, axis='y')
    for i, v in enumerate(pr_values):
        axes[2].text(i, v + 0.02, f'{v:.3f}', ha='center', fontsize=10, fontweight='bold')

    # 4. F1-Score
    f1_score = metrics.get('f1_score', 0)
    axes[3].bar(['F1-Score'], [f1_score], color='#f39c12', alpha=0.8, edgecolor='black')
    axes[3].set_ylabel('Score', fontsize=12)
    axes[3].set_title('F1-Score', fontsize=14)
    axes[3].set_ylim([0, 1])
    axes[3].grid(True, alpha=0.3, axis='y')
    axes[3].text(0, f1_score + 0.02, f'{f1_score:.3f}', ha='center', fontsize=10, fontweight='bold')

    # 5. Learning Rate Schedule
    axes[4].plot(epochs, history['learning_rate'], linewidth=2, color='#16a085')
    axes[4].set_xlabel('Epoch', fontsize=12)
    axes[4].set_ylabel('Learning Rate', fontsize=12)
    axes[4].set_title('Learning Rate Schedule', fontsize=14)
    axes[4].set_yscale('log')
    axes[4].grid(True, alpha=0.3)

    # 6. Summary text
    axes[5].axis('off')
    summary_text = f"""
EXPERIMENT SUMMARY
{'='*35}

Model Family: Faster R-CNN
Best Epoch: {best_epoch}
Best Val Loss: {min(history['val_loss']):.4f}

METRICS
{'='*35}
mAP@0.5:      {metrics.get('mAP_50', 0):.4f}
mAP@0.5:0.95: {metrics.get('mAP_50_95', 0):.4f}
mAP@0.75:     {metrics.get('mAP_75', 0):.4f}
Precision:    {metrics.get('precision', 0):.4f}
Recall:       {metrics.get('recall', 0):.4f}
F1-Score:     {metrics.get('f1_score', 0):.4f}

PERFORMANCE
{'='*35}
Inference:    {metrics.get('inference_time_ms', 0):.2f} ms/img
Parameters:   {metrics.get('total_params_M', 0):.2f}M
"""
    axes[5].text(0.1, 0.5, summary_text, fontsize=11, family='monospace',
                verticalalignment='center', bbox=dict(boxstyle='round',
                facecolor='wheat', alpha=0.3))

    plt.tight_layout(rect=[0, 0, 1, 0.96])

    # Create plots directory if it doesn't exist
    plots_dir = experiment_dir / 'plots'
    plots_dir.mkdir(parents=True, exist_ok=True)

    plt.savefig(plots_dir / 'comprehensive_results.png',
                dpi=300, bbox_inches='tight')
    plt.close()

    print(f"[OK] Comprehensive plots saved to: {plots_dir / 'comprehensive_results.png'}")



In [7]:
!pwd

/home/jupyter-st124895/cv_project


In [8]:
# ==============================================================================
# SECTION 5: Dataset Paths & Configuration
# ==============================================================================

# Google Drive paths
# EXPERIMENTS_PATH = '/content/drive/MyDrive/cv_project/04_experiments'
# DATASET_PATH = '/content/drive/MyDrive/cv_project/03_datasets/oil_palm_coco_v1'
# Puffer Paths
EXPERIMENTS_PATH = '/home/jupyter-st124895/cv_project/04_experiments'
DATASET_PATH =     '/home/jupyter-st124895/cv_project/03_datasets/oil_palm_coco_v2'

# Create experiments directory
Path(EXPERIMENTS_PATH).mkdir(parents=True, exist_ok=True)

# Verify dataset paths
dataset_path = Path(DATASET_PATH)
train_images_dir = dataset_path / 'train'
val_images_dir = dataset_path / 'valid'
# train_ann_file = dataset_path / 'annotations' / 'instances_train.json'
# val_ann_file = dataset_path / 'annotations' / 'instances_val.json'

train_ann_file = dataset_path / 'train' / '_annotations.coco.json'
val_ann_file = dataset_path / 'valid' /  '_annotations.coco.json'


print(f"[INFO] Dataset verification:")
print(f"  Train images: {train_images_dir.exists()} - {train_images_dir}")
print(f"  Val images: {val_images_dir.exists()} - {val_images_dir}")
print(f"  Train annotations: {train_ann_file.exists()} - {train_ann_file}")
print(f"  Val annotations: {val_ann_file.exists()} - {val_ann_file}")

if not all([train_images_dir.exists(), val_images_dir.exists(),
            train_ann_file.exists(), val_ann_file.exists()]):
    print("\n[ERROR] Dataset not found! Please upload dataset to Google Drive.")
else:
    print("\n[OK] Dataset ready!")

[INFO] Dataset verification:
  Train images: True - /home/jupyter-st124895/cv_project/03_datasets/oil_palm_coco_v2/train
  Val images: True - /home/jupyter-st124895/cv_project/03_datasets/oil_palm_coco_v2/valid
  Train annotations: True - /home/jupyter-st124895/cv_project/03_datasets/oil_palm_coco_v2/train/_annotations.coco.json
  Val annotations: True - /home/jupyter-st124895/cv_project/03_datasets/oil_palm_coco_v2/valid/_annotations.coco.json

[OK] Dataset ready!


In [9]:
# ==============================================================================
# SECTION 6: Run Experiments
# ==============================================================================
"""
CONFIGURABLE ARCHITECTURE PARAMETERS (Opción A):

You can now configure these architecture parameters directly in experiments_config:

- box_score_thresh (float, default 0.05): Minimum confidence threshold for detections
  * Higher values (0.10-0.15) reduce false positives but may miss low-confidence objects
  * Recommended range: 0.05-0.20

- box_nms_thresh (float, default 0.5): NMS IoU threshold
  * Lower values (0.4-0.45) allow closer boxes (good for dense scenes)
  * Higher values (0.5-0.6) suppress more overlapping boxes
  * Recommended range: 0.4-0.6

- box_detections_per_img (int, default 100): Maximum detections per image
  * Your dataset averages 44 objects/image, so 80-120 is reasonable
  * Recommended range: 60-150

- rpn_fg_iou_thresh (float, default 0.7): RPN foreground IoU threshold
  * Lower values (0.6) are more lenient for dense/overlapping objects
  * Recommended range: 0.6-0.7

- rpn_bg_iou_thresh (float, default 0.3): RPN background IoU threshold
  * Higher values (0.4) are more strict about background classification
  * Recommended range: 0.3-0.5

- box_positive_fraction (float, default 0.25): Ratio of positive samples in ROI head
  * Higher values (0.30-0.35) provide more positive examples for dense scenes
  * Recommended range: 0.20-0.35

EXAMPLE USAGE in experiments_config:
{
    'name': 'optimized_score_thresh',
    'description': 'Test higher confidence threshold',
    'backbone': 'resnet50',
    'pretrained': True,
    'num_epochs': 50,
    'batch_size': 4,
    'learning_rate': 0.005,
    ... (standard params) ...

    # NEW: Architecture parameters
    'box_score_thresh': 0.10,        # Higher confidence
    'box_nms_thresh': 0.5,           # Keep default
    'rpn_fg_iou_thresh': 0.6,        # More lenient
    'box_positive_fraction': 0.30,   # More positive samples
}

NOTE: If parameter is not specified, torchvision defaults are used.
All parameters are tracked in CSV for analysis.
"""

# Device detection
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

print(f"\n[INFO] Using device: {device}")

# Experiment configuration with data augmentation
experiments_config = [
    # ==========================================================================
    # FASE 1: ResNet101 - Validación inicial
    # ==========================================================================
    {
        'name': 'resnet101_baseline',
        'description': 'ResNet101 con configuración baseline para comparación',
        'backbone': 'resnet101',
        'pretrained': True,
        'num_epochs': 110,
        'batch_size': 4,
        'learning_rate': 0.004,
        'momentum': 0.9,
        'weight_decay': 0.0005,
        'step_size': 20,
        'gamma': 0.1,
        'patience': 35,
        # Augmentation probada que funciona bien
        'hsv_h': 0.015, 'hsv_s': 0.08, 'hsv_v': 0.08,
        'degrees': 4.0, 'translate': 0.04, 'scale': 0.08,
        'horizontal_flip': 0.5, 'vertical_flip': 0.0,
        'blur': False, 'brightness_contrast': True
        # NO architecture params = defaults
    },

    # # ==========================================================================
    # # FASE 2: ResNet101 + Mejores parámetros encontrados
    # # ==========================================================================
    # {
    #     'name': 'resnet101_best_params',
    #     'description': 'ResNet101 + score 0.10 + pos_frac 0.30 (mejor de ResNet50)',
    #     'backbone': 'resnet101',
    #     'pretrained': True,
    #     'num_epochs': 110,
    #     'batch_size': 4,
    #     'learning_rate': 0.004,
    #     'momentum': 0.9,
    #     'weight_decay': 0.0005,
    #     'step_size': 20,
    #     'gamma': 0.1,
    #     'patience': 35,
    #     'hsv_h': 0.015, 'hsv_s': 0.08, 'hsv_v': 0.08,
    #     'degrees': 4.0, 'translate': 0.04, 'scale': 0.08,
    #     'horizontal_flip': 0.5, 'vertical_flip': 0.0,
    #     'blur': False, 'brightness_contrast': True,
    #     # ARCHITECTURE - Best params from ResNet50:
    #     'box_score_thresh': 0.10,
    #     'box_positive_fraction': 0.30
    # },

    # # ==========================================================================
    # # FASE 3: ResNet101 + Longer Training
    # # ==========================================================================
    # {
    #     'name': 'resnet101_long_training',
    #     'description': 'ResNet101 + 200 epochs + decay suave + best params',
    #     'backbone': 'resnet101',
    #     'pretrained': True,
    #     'num_epochs': 200,
    #     'batch_size': 4,
    #     'learning_rate': 0.004,
    #     'momentum': 0.9,
    #     'weight_decay': 0.0005,
    #     'step_size': 30,  # Decay más lento
    #     'gamma': 0.2,     # Menos agresivo
    #     'patience': 60,   # Más paciencia
    #     'hsv_h': 0.015, 'hsv_s': 0.08, 'hsv_v': 0.08,
    #     'degrees': 4.0, 'translate': 0.04, 'scale': 0.08,
    #     'horizontal_flip': 0.5, 'vertical_flip': 0.0,
    #     'blur': False, 'brightness_contrast': True,
    #     # ARCHITECTURE:
    #     'box_score_thresh': 0.10,
    #     'box_positive_fraction': 0.30
    # },

    # # ==========================================================================
    # # FASE 4: ResNet101 + Strong Augmentation
    # # ==========================================================================
    # {
    #     'name': 'resnet101_strong_aug',
    #     'description': 'ResNet101 + augmentation más fuerte para mejor generalización',
    #     'backbone': 'resnet101',
    #     'pretrained': True,
    #     'num_epochs': 150,
    #     'batch_size': 4,
    #     'learning_rate': 0.004,
    #     'momentum': 0.9,
    #     'weight_decay': 0.0005,
    #     'step_size': 25,
    #     'gamma': 0.1,
    #     'patience': 45,
    #     # STRONG AUGMENTATION:
    #     'hsv_h': 0.03, 'hsv_s': 0.15, 'hsv_v': 0.15,
    #     'degrees': 8.0, 'translate': 0.1, 'scale': 0.15,
    #     'horizontal_flip': 0.5, 'vertical_flip': 0.3,
    #     'blur': True, 'brightness_contrast': True,
    #     # ARCHITECTURE:
    #     'box_score_thresh': 0.10,
    #     'box_positive_fraction': 0.30
    # },

    # # ==========================================================================
    # # FASE 5: ResNet101 + NMS Tuning
    # # ==========================================================================
    # {
    #     'name': 'resnet101_nms_06',
    #     'description': 'ResNet101 + NMS 0.6 (más tolerante para dense objects)',
    #     'backbone': 'resnet101',
    #     'pretrained': True,
    #     'num_epochs': 120,
    #     'batch_size': 4,
    #     'learning_rate': 0.004,
    #     'momentum': 0.9,
    #     'weight_decay': 0.0005,
    #     'step_size': 25,
    #     'gamma': 0.1,
    #     'patience': 40,
    #     'hsv_h': 0.015, 'hsv_s': 0.08, 'hsv_v': 0.08,
    #     'degrees': 4.0, 'translate': 0.04, 'scale': 0.08,
    #     'horizontal_flip': 0.5, 'vertical_flip': 0.0,
    #     'blur': False, 'brightness_contrast': True,
    #     # ARCHITECTURE:
    #     'box_score_thresh': 0.10,
    #     'box_nms_thresh': 0.6,  # Más tolerante
    #     'box_positive_fraction': 0.30
    # },

    # {
    #     'name': 'resnet101_nms_065',
    #     'description': 'ResNet101 + NMS 0.65 (balance)',
    #     'backbone': 'resnet101',
    #     'pretrained': True,
    #     'num_epochs': 120,
    #     'batch_size': 4,
    #     'learning_rate': 0.004,
    #     'momentum': 0.9,
    #     'weight_decay': 0.0005,
    #     'step_size': 25,
    #     'gamma': 0.1,
    #     'patience': 40,
    #     'hsv_h': 0.015, 'hsv_s': 0.08, 'hsv_v': 0.08,
    #     'degrees': 4.0, 'translate': 0.04, 'scale': 0.08,
    #     'horizontal_flip': 0.5, 'vertical_flip': 0.0,
    #     'blur': False, 'brightness_contrast': True,
    #     # ARCHITECTURE:
    #     'box_score_thresh': 0.10,
    #     'box_nms_thresh': 0.65,
    #     'box_positive_fraction': 0.30
    # },

    # # ==========================================================================
    # # FASE 6: ResNet101 + All Optimizations
    # # ==========================================================================
    # {
    #     'name': 'resnet101_all_optimizations',
    #     'description': 'ResNet101 + TODO lo mejor encontrado',
    #     'backbone': 'resnet101',
    #     'pretrained': True,
    #     'num_epochs': 200,
    #     'batch_size': 4,
    #     'learning_rate': 0.004,
    #     'momentum': 0.9,
    #     'weight_decay': 0.0005,
    #     'step_size': 30,
    #     'gamma': 0.2,
    #     'patience': 60,
    #     # Moderate augmentation (balance)
    #     'hsv_h': 0.02, 'hsv_s': 0.1, 'hsv_v': 0.1,
    #     'degrees': 6.0, 'translate': 0.06, 'scale': 0.12,
    #     'horizontal_flip': 0.5, 'vertical_flip': 0.2,
    #     'blur': False, 'brightness_contrast': True,
    #     # ALL ARCHITECTURE OPTIMIZATIONS:
    #     'box_score_thresh': 0.10,
    #     'box_nms_thresh': 0.6,
    #     'box_positive_fraction': 0.30
    # },

    # # ==========================================================================
    # # BONUS: ResNet101 + Cosine-like LR Schedule
    # # ==========================================================================
    # {
    #     'name': 'resnet101_cosine_lr',
    #     'description': 'ResNet101 + cosine-like learning rate (smooth decay)',
    #     'backbone': 'resnet101',
    #     'pretrained': True,
    #     'num_epochs': 150,
    #     'batch_size': 4,
    #     'learning_rate': 0.004,
    #     'momentum': 0.9,
    #     'weight_decay': 0.0005,
    #     'step_size': 5,   # Pequeño step
    #     'gamma': 0.95,    # Decay suave (simula cosine)
    #     'patience': 50,
    #     'hsv_h': 0.015, 'hsv_s': 0.08, 'hsv_v': 0.08,
    #     'degrees': 4.0, 'translate': 0.04, 'scale': 0.08,
    #     'horizontal_flip': 0.5, 'vertical_flip': 0.0,
    #     'blur': False, 'brightness_contrast': True,
    #     # ARCHITECTURE:
    #     'box_score_thresh': 0.10,
    #     'box_positive_fraction': 0.30
    # },
]


# Run experiments
for exp_config in experiments_config:
    print(f"\n{'='*80}")
    print(f"  STARTING EXPERIMENT: {exp_config['name']}")
    print(f"  {exp_config['description']}")
    print(f"{'='*80}")

    # Create trainer
    trainer = FasterRCNNTrainer(
        experiment_name=exp_config['name'],
        model_family_dir='02_faster_rcnn'
    )

    # Create transforms with augmentation
    train_transforms = get_train_transforms(exp_config)
    val_transforms = get_val_transforms()

    print(f"\n[INFO] Data augmentation settings:")
    print(f"  HSV: h={exp_config.get('hsv_h', 0)}, s={exp_config.get('hsv_s', 0)}, v={exp_config.get('hsv_v', 0)}")
    print(f"  Geometric: degrees={exp_config.get('degrees', 0)}, translate={exp_config.get('translate', 0)}, scale={exp_config.get('scale', 0)}")
    print(f"  Flip: horizontal={exp_config.get('horizontal_flip', 0)}, vertical={exp_config.get('vertical_flip', 0)}")
    print(f"  Other: blur={exp_config.get('blur', False)}, brightness_contrast={exp_config.get('brightness_contrast', False)}")

    # Create datasets
    train_dataset = COCODataset(
        root_dir=train_images_dir,
        annotation_file=train_ann_file,
        transforms=train_transforms
    )

    val_dataset = COCODataset(
        root_dir=val_images_dir,
        annotation_file=val_ann_file,
        transforms=val_transforms
    )

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=exp_config['batch_size'],
        shuffle=True,
        num_workers=2,
        collate_fn=lambda x: tuple(zip(*x))
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=exp_config['batch_size'],
        shuffle=False,
        num_workers=2,
        collate_fn=lambda x: tuple(zip(*x))
    )

    # Create model (num_classes = 1 (palm) + 1 (background) = 2)
    model = get_model(
        num_classes=2,
        backbone=exp_config['backbone'],
        pretrained=exp_config['pretrained'],
        # Configurable architecture parameters (Opción A)
        box_score_thresh=exp_config.get('box_score_thresh'),
        box_nms_thresh=exp_config.get('box_nms_thresh'),
        box_detections_per_img=exp_config.get('box_detections_per_img'),
        rpn_fg_iou_thresh=exp_config.get('rpn_fg_iou_thresh'),
        rpn_bg_iou_thresh=exp_config.get('rpn_bg_iou_thresh'),
        box_positive_fraction=exp_config.get('box_positive_fraction')
    )

    # Create optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(
        params,
        lr=exp_config['learning_rate'],
        momentum=exp_config['momentum'],
        weight_decay=exp_config['weight_decay']
    )

    # Learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer,
        step_size=exp_config['step_size'],
        gamma=exp_config['gamma']
    )

    # Train model
    history, best_val_loss, best_epoch = trainer.train(
        train_loader=train_loader,
        val_loader=val_loader,
        model=model,
        optimizer=optimizer,
        lr_scheduler=lr_scheduler,
        num_epochs=exp_config['num_epochs'],
        device=device,
        config=exp_config
    )

    print(f"\n[OK] Training completed for {exp_config['name']}!")
    print(f"  Results saved to: {trainer.experiment_dir}")

    # Automatic evaluation and CSV logging (same as YOLOv8)
    print(f"\n[INFO] Evaluating model and saving metrics...")

    # Load best model for evaluation (must match training config)
    model_eval = get_model(
        num_classes=2,
        backbone=exp_config['backbone'],
        pretrained=False,
        # Must use same architecture parameters as training
        box_score_thresh=exp_config.get('box_score_thresh'),
        box_nms_thresh=exp_config.get('box_nms_thresh'),
        box_detections_per_img=exp_config.get('box_detections_per_img'),
        rpn_fg_iou_thresh=exp_config.get('rpn_fg_iou_thresh'),
        rpn_bg_iou_thresh=exp_config.get('rpn_bg_iou_thresh'),
        box_positive_fraction=exp_config.get('box_positive_fraction')
    )
    model_eval.load_state_dict(torch.load(trainer.weights_dir / 'best.pt'))
    model_eval.to(device)

    # Evaluate on validation set
    metrics = evaluate_model_coco(model_eval, val_loader, device, val_ann_file)

    # Get dataset info
    with open(train_ann_file, 'r') as f:
        train_data = json.load(f)
    with open(val_ann_file, 'r') as f:
        val_data = json.load(f)

    dataset_info = {
        'num_train_images': len(train_data['images']),
        'num_val_images': len(val_data['images']),
        'num_train_boxes': len(train_data['annotations']),
        'num_val_boxes': len(val_data['annotations'])
    }

    # Save metrics to CSV (master + family)
    training_time_seconds = trainer.training_time
    save_metrics_to_csv(
        experiment_dir=trainer.experiment_dir,
        experiment_id=trainer.experiment_id,
        experiment_name=exp_config['name'],
        config=exp_config,
        metrics=metrics,
        dataset_info=dataset_info,
        training_time=training_time_seconds,
        best_epoch=best_epoch
    )

    # Update best model tracker (overall + family)
    update_best_model_tracker(
        experiment_id=trainer.experiment_id,
        experiment_name=exp_config['name'],
        metrics=metrics,
        config=exp_config
    )

    # Create comprehensive plots
    history_plots = {
        'train_loss': history['train_loss'],
        'val_loss': history['val_loss'],
        'learning_rate': history['learning_rate']
    }
    create_comprehensive_plots(
        experiment_dir=trainer.experiment_dir,
        history=history_plots,
        metrics=metrics,
        best_epoch=best_epoch,
        experiment_id=trainer.experiment_id
    )

    print(f"\n[OK] Metrics saved to CSVs and best model tracker updated!")
    print(f"  mAP@0.5:      {metrics['mAP_50']:.4f}")
    print(f"  mAP@0.5:0.95: {metrics['mAP_50_95']:.4f}")



[INFO] Using device: cuda

  STARTING EXPERIMENT: resnet101_baseline
  ResNet101 con configuración baseline para comparación

  EXPERIMENT: exp_074_resnet101_baseline
  Family: 02_faster_rcnn
  Directory: /home/jupyter-st124895/cv_project/04_experiments/02_faster_rcnn/exp_074_resnet101_baseline


[INFO] Data augmentation settings:
  HSV: h=0.015, s=0.08, v=0.08
  Geometric: degrees=4.0, translate=0.04, scale=0.08
  Flip: horizontal=0.5, vertical=0.0
  Other: blur=False, brightness_contrast=True


/home/jupyter-st124895/.local/lib/python3.12/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/home/jupyter-st124895/.local/lib/python3.12/site-packages/albumentations/core/composition.py:331: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


[OK] Loaded 306 images
[OK] Loaded 39465 annotations
[OK] Loaded 153 images
[OK] Loaded 19666 annotations
[INFO] Starting training for 110 epochs...
[INFO] Device: cuda

[EPOCH 1/110]
  Batch [10/77] - Loss: 1.5123
  Batch [20/77] - Loss: 1.1444
  Batch [30/77] - Loss: 1.0704
  Batch [40/77] - Loss: 0.8621
  Batch [50/77] - Loss: 0.7592
  Batch [60/77] - Loss: 0.6721
  Batch [70/77] - Loss: 0.6700
  Train Loss: 1.0183
  Val Loss: 0.6762
  Time: 51.60s
  ✓ Best model saved (val_loss: 0.6762)

[EPOCH 2/110]
  Batch [10/77] - Loss: 0.6456
  Batch [20/77] - Loss: 0.5969
  Batch [30/77] - Loss: 0.6148
  Batch [40/77] - Loss: 0.6458
  Batch [50/77] - Loss: 0.6389
  Batch [60/77] - Loss: 0.7468
  Batch [70/77] - Loss: 0.6070
  Train Loss: 0.6778
  Val Loss: 0.6263
  Time: 51.29s
  ✓ Best model saved (val_loss: 0.6263)

[EPOCH 3/110]
  Batch [10/77] - Loss: 0.7109
  Batch [20/77] - Loss: 0.6782
  Batch [30/77] - Loss: 0.6019
  Batch [40/77] - Loss: 0.5992
  Batch [50/77] - Loss: 0.6959
  Batch

/home/jupyter-st124895/.local/lib/python3.12/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/home/jupyter-st124895/.local/lib/python3.12/site-packages/albumentations/core/composition.py:331: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


[INFO] Starting training for 110 epochs...
[INFO] Device: cuda

[EPOCH 1/110]
  Batch [10/77] - Loss: 1.8788
  Batch [20/77] - Loss: 1.2573
  Batch [30/77] - Loss: 1.1755
  Batch [40/77] - Loss: 0.8892
  Batch [50/77] - Loss: 0.7640
  Batch [60/77] - Loss: 0.6898
  Batch [70/77] - Loss: 0.6480
  Train Loss: 1.1087
  Val Loss: 0.6086
  Time: 51.01s
  ✓ Best model saved (val_loss: 0.6086)

[EPOCH 2/110]
  Batch [10/77] - Loss: 0.6406
  Batch [20/77] - Loss: 0.7107
  Batch [30/77] - Loss: 0.6623
  Batch [40/77] - Loss: 0.6476
  Batch [50/77] - Loss: 0.6494
  Batch [60/77] - Loss: 0.5553
  Batch [70/77] - Loss: 0.6684
  Train Loss: 0.6588
  Val Loss: 0.6244
  Time: 50.50s

[EPOCH 3/110]
  Batch [10/77] - Loss: 0.6419
  Batch [20/77] - Loss: 0.6968
  Batch [30/77] - Loss: 0.6994
  Batch [40/77] - Loss: 0.6125
  Batch [50/77] - Loss: 0.6638
  Batch [60/77] - Loss: 0.5918
  Batch [70/77] - Loss: 0.5851
  Train Loss: 0.6307
  Val Loss: 0.5423
  Time: 51.06s
  ✓ Best model saved (val_loss: 0.54

KeyboardInterrupt: 

In [10]:
!nvidia-smi

Thu Nov 20 16:14:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     Off |   00000000:84:00.0 Off |                  N/A |
| 43%   59C    P8             37W /  250W |   10764MiB /  11264MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
# ==============================================================================
# SECTION 7: Re-evaluate Trained Models (OPTIONAL)
# ==============================================================================
# NOTE: Evaluation is now done automatically after training in SECTION 6.
# This section is OPTIONAL and only needed if you want to re-evaluate
# an already trained model (e.g., after changing evaluation parameters).

def evaluate_experiment(experiment_id, backbone='resnet50'):
    """
    Evaluate a trained Faster R-CNN experiment and save metrics to CSV.

    Args:
        experiment_id: Experiment ID (e.g., 'exp_001_baseline_resnet50')
        backbone: Model backbone used ('resnet50' or 'resnet101')

    Returns:
        dict: Evaluation metrics

    Note: Architecture parameters are loaded from experiment_log.json config
    """
    print(f"\n{'='*70}")
    print(f"  EVALUATING: {experiment_id}")
    print(f"{'='*70}\n")

    experiment_dir = Path(EXPERIMENTS_PATH) / '02_faster_rcnn' / experiment_id

    # Check if experiment exists
    if not experiment_dir.exists():
        print(f"[ERROR] Experiment not found: {experiment_dir}")
        return None

    # Load experiment config first to get architecture parameters
    print("[INFO] Loading experiment config...")
    with open(experiment_dir / 'experiment_log.json', 'r') as f:
        exp_log = json.load(f)

    exp_config = exp_log.get('config', {})

    # Load best model with same architecture parameters as training
    print("[INFO] Loading best model...")
    model = get_model(
        num_classes=2,
        backbone=backbone,
        pretrained=False,
        # Use same architecture parameters as training
        box_score_thresh=exp_config.get('box_score_thresh'),
        box_nms_thresh=exp_config.get('box_nms_thresh'),
        box_detections_per_img=exp_config.get('box_detections_per_img'),
        rpn_fg_iou_thresh=exp_config.get('rpn_fg_iou_thresh'),
        rpn_bg_iou_thresh=exp_config.get('rpn_bg_iou_thresh'),
        box_positive_fraction=exp_config.get('box_positive_fraction')
    )
    model.load_state_dict(torch.load(experiment_dir / 'weights' / 'best.pt'))
    model.to(device)

    # Create validation loader
    print("[INFO] Loading validation dataset...")
    val_dataset = COCODataset(val_images_dir, val_ann_file, get_val_transforms())
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False,
                            num_workers=0, collate_fn=lambda x: tuple(zip(*x)))

    # Evaluate on validation set
    metrics = evaluate_model_coco(model, val_loader, device, val_ann_file)

    # Get dataset info from annotation files
    with open(train_ann_file, 'r') as f:
        train_data = json.load(f)
    with open(val_ann_file, 'r') as f:
        val_data = json.load(f)

    dataset_info = {
        'num_train_images': len(train_data['images']),
        'num_val_images': len(val_data['images']),
        'num_train_boxes': len(train_data['annotations']),
        'num_val_boxes': len(val_data['annotations'])
    }

    # Save metrics to CSV
    print("\n[INFO] Saving metrics to CSV...")
    # training_time_minutes from exp_log is already in minutes, multiply by 60 to get seconds
    training_time_seconds = exp_log.get('training_time_minutes', 0) * 60
    save_metrics_to_csv(
        experiment_dir=experiment_dir,
        experiment_id=experiment_id,
        experiment_name=exp_log['config'].get('name', experiment_id),
        config=exp_log['config'],
        metrics=metrics,
        dataset_info=dataset_info,
        training_time=training_time_seconds,  # Now in seconds, will be converted to hours in CSV
        best_epoch=exp_log['best_epoch']
    )

    # Update best model tracker
    update_best_model_tracker(
        experiment_id=experiment_id,
        experiment_name=exp_log['config'].get('name', experiment_id),
        metrics=metrics,
        config=exp_log['config']
    )

    # Create comprehensive plots (6 subplots like YOLOv8)
    print("\n[INFO] Creating comprehensive plots...")
    history = {
        'train_loss': exp_log.get('train_loss', []),
        'val_loss': exp_log.get('val_loss', []),
        'learning_rate': exp_log.get('learning_rate', [])
    }
    create_comprehensive_plots(
        experiment_dir=experiment_dir,
        history=history,
        metrics=metrics,
        best_epoch=exp_log['best_epoch'],
        experiment_id=experiment_id
    )

    # Print summary
    print(f"\n{'='*70}")
    print(f"  EVALUATION SUMMARY - {experiment_id}")
    print(f"{'='*70}")
    print(f"  mAP@0.5:      {metrics['mAP_50']:.4f}")
    print(f"  mAP@0.5:0.95: {metrics['mAP_50_95']:.4f}")
    print(f"  mAP@0.75:     {metrics['mAP_75']:.4f}")
    print(f"  Precision:    {metrics['precision']:.4f}")
    print(f"  Recall:       {metrics['recall']:.4f}")
    print(f"  F1-Score:     {metrics['f1_score']:.4f}")
    print(f"  Inference:    {metrics['inference_time_ms']:.2f} ms/image")
    print(f"  Parameters:   {metrics['total_params_M']:.2f}M")
    print(f"{'='*70}\n")

    return metrics


# Example usage (uncomment to evaluate):
"""
# Evaluate all completed experiments
experiment_ids = [
    'exp_001_baseline_resnet50',
    'exp_002_augmented_resnet50',
    'exp_003_heavy_aug_resnet50'
]

for exp_id in experiment_ids:
    evaluate_experiment(exp_id, backbone='resnet50')
"""

print("\n[INFO] SECTION 7 loaded (OPTIONAL)")
print("Use: evaluate_experiment('exp_001_baseline_resnet50') to re-evaluate a trained model")



[INFO] SECTION 7 loaded (OPTIONAL)
Use: evaluate_experiment('exp_001_baseline_resnet50') to re-evaluate a trained model

  Oil Palm Detection - Faster R-CNN Training System
  All sections loaded successfully!


In [8]:
# Evaluate all completed experiments
experiment_ids = [
    'exp_001_baseline_resnet50',
    'exp_002_augmented_resnet50',
    'exp_003_heavy_aug_resnet50'
]

for exp_id in experiment_ids:
    evaluate_experiment(exp_id, backbone='resnet50')


  EVALUATING: exp_001_baseline_resnet50

[INFO] Loading best model...
[INFO] Loading validation dataset...
[OK] Loaded 13 images
[OK] Loaded 546 annotations
[INFO] Running inference for COCO evaluation...


/usr/local/lib/python3.12/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


[OK] Generated 1098 predictions
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!

[INFO] Running COCO evaluation...
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.89s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.525
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.783
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.686
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.525
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.017
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.1

/usr/local/lib/python3.12/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


[OK] Generated 1190 predictions
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!

[INFO] Running COCO evaluation...
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.21s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.521
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.786
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.653
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.522
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.017
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.1

/usr/local/lib/python3.12/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


[OK] Generated 1216 predictions
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!

[INFO] Running COCO evaluation...
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.22s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.528
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.778
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.695
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.529
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.017
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.1